# Evaluación 2 – Data Science para la Energía Solar

Alumna: Nicole Torres

Prof: Cristobal Parrado

## 1. Introducción
Este notebook presenta el desarrollo completo de la Evaluación 2 del curso Data Science para la Energía Solar.
Se evalúa técnica y económicamente el desempeño de una planta fotovoltaica de 50 MWDC en tres localidades:
Calama, Salvador y Vallenar, usando datos TMY. Se incluyen limpieza de datos, simulaciones
con PySAM, análisis de LCOE y VAN, estudio de sensibilidad y un dashboard interactivo.

In [9]:
# =============================================================
#  DESCRIPCIÓN E IMPORTACIÓN DE LIBRERÍAS
# =============================================================
#
# Autor: Nicole Torres
# Descripción: Este script procesa, limpia y analiza archivos de datos horarios solares (GHI, DNI, DHI)
#              para diferentes localidades, generando archivos TMY artificiales, reportes EDA y gráficos.
#              El flujo incluye transformación, limpieza avanzada, reconstrucción de fechas originales,
#              llenado jerárquico de NaNs y generación de informes comparativos.
#
# Uso:
#   1. Coloca los archivos *_corrupted.csv en la carpeta del proyecto.
#   2. Ejecuta este script:
#        python3 limpieza.py
#   3. Los resultados se guardarán en las carpetas 'datos_limpios/', 'reports/' y 'plots/'.
#
# Requiere: pandas, numpy, matplotlib, pvlib, PySAM
# =============================================================
# =============================
# 📦 Importación de librerías
# =============================
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import numpy as np
import pvlib
import os
from pvlib.solarposition import get_solarposition
import PySAM.Pvwattsv7 as pv
import PySAM.Lcoefcr as Lcoefcr


# === CONFIGURACIÓN DE DIRECTORIOS ===
DATOS_LIMPIOS_DIR = Path('datos_limpios')
DATOS_LIMPIOS_DIR.mkdir(exist_ok=True)
RESULTADOS_PV_DIR = Path('resultados_pv')
RESULTADOS_PV_DIR.mkdir(exist_ok=True)

## 2. Carga y limpieza de datos

Se trabajó con archivos meteorológicos por localidad (Calama, Salvador, Vallenar), cada uno con 43.824 registros horarios de un archivo TMY. Aproximadamente un 8 % de los datos estaban corruptos, incluyendo:

- Valores nulos (NaN), ceros, y negativos en columnas clave como GHI, DNI, DHI, TempC y Wind_mps.
- Outliers detectados según umbrales físicos razonables.

La estrategia aplicada consistió en:

1. Cargar los archivos y evaluar calidad de datos.
2. Reemplazar valores negativos y nulos con NaN.
3. Aplicar interpolación lineal cuando los huecos eran menores o iguales a 3 pasos.
4. Validar la limpieza con gráficas comparativas antes/después y estadísticas básicas.

El objetivo fue mantener los 43.824 registros por archivo, asegurando consistencia temporal y viabilidad física de los datos para simulación.


In [7]:
# =============================================================
#  LIMPIEZA Y ANÁLISIS DE DATOS SOLARES TMY PARA SIMULACIÓN PV
# =============================================================

pd.set_option('future.no_silent_downcasting', True)

# =============================================================
# FUNCIONES PRINCIPALES DEL FLUJO DE LIMPIEZA Y ANÁLISIS
# =============================================================

def transformar_a_tmy_con_metadatos(csv_path, output_path, metadata_dict):
    """
    Transforma un archivo CSV con datos horarios en un archivo TMY artificial con metadatos en el encabezado.
    - Ordena cronológicamente, recorta a 8760 filas, asigna año artificial y reordena columnas.
    - Escribe las tres primeras líneas como metadatos y encabezados.
    """
    # Leer el archivo CSV
    df = pd.read_csv(csv_path)

    # Eliminar columna 'datetime' si existe
    if 'datetime' in df.columns:
        df = df.drop(columns=['datetime'])

    # Ordenar cronológicamente y recortar o completar a 8760 filas
    df = df.sort_values(by=["Month", "Day", "Hour", "Minute"])
    df = df.reset_index(drop=True)
    df = df.iloc[:8760]  # en caso de que tenga más filas

    # Asignar un año artificial constante
    df['Year'] = 1990

    # Reordenar columnas: primero las de fecha
    columnas_fecha = ['Year', 'Month', 'Day', 'Hour', 'Minute']
    otras = [c for c in df.columns if c not in columnas_fecha]
    df = df[columnas_fecha + otras]

    # Escribir archivo con las tres primeras líneas de metadatos
    with open(output_path, "w", encoding="utf-8") as f:
        # Línea 1: encabezados de metadatos
        f.write("Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation\n")
        # Línea 2: valores de metadatos
        f.write(",".join(str(metadata_dict[k]) for k in [
            "Source", "Location ID", "City", "State", "Country",
            "Latitude", "Longitude", "Time Zone", "Elevation"
        ]) + "\n")
        # Línea 3: encabezado de columnas de datos
        f.write(",".join(df.columns) + "\n")
        # Resto de datos
        df.to_csv(f, index=False, header=False)

    return df


def plot_tmy_data(df, location, output_dir="plots", is_clean=False):
    """
    Genera y guarda gráficos de GHI, DNI y DHI para una localidad.
    - Los gráficos se guardan en la carpeta 'plots/'.
    """
    # Crear directorio para gráficos si no existe
    Path(output_dir).mkdir(exist_ok=True)

    # Configurar estilo de gráficos
    plt.style.use('default')

    # Crear figura con tres subplots
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 15))
    title_suffix = " (Datos Limpios)" if is_clean else ""
    fig.suptitle(f'Radiación Solar por Hora - {location}{title_suffix}', fontsize=16, y=0.95)

    # Crear fechas para el eje x
    dates = [datetime(1990, int(row['Month']), int(row['Day']), int(row['Hour'])) 
             for _, row in df.iterrows()]

    # Graficar cada componente en su propio subplot
    ax1.plot(dates, df['GHI'], 'b-', linewidth=1, alpha=0.7)
    ax1.set_title('Radiación Global Horizontal (GHI)', fontsize=12)
    ax1.set_ylabel('GHI (W/m²)', fontsize=10)
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 1300)  # Establecer límite superior para GHI

    ax2.plot(dates, df['DNI'], 'r-', linewidth=1, alpha=0.7)
    ax2.set_title('Radiación Normal Directa (DNI)', fontsize=12)
    ax2.set_ylabel('DNI (W/m²)', fontsize=10)
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, 1300)  # Establecer límite superior para DNI

    ax3.plot(dates, df['DHI'], 'g-', linewidth=1, alpha=0.7)
    ax3.set_title('Radiación Horizontal Difusa (DHI)', fontsize=12)
    ax3.set_ylabel('DHI (W/m²)', fontsize=10)
    ax3.set_xlabel('Mes', fontsize=10)
    ax3.grid(True, alpha=0.3)
    ax3.set_ylim(0, 600)  # Establecer límite superior para DHI

    # Configurar formato del eje x para todos los subplots
    for ax in [ax1, ax2, ax3]:
        ax.xaxis.set_major_locator(mdates.MonthLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

    # Ajustar layout y guardar
    plt.tight_layout()
    suffix = "_clean" if is_clean else ""
    plt.savefig(f'{output_dir}/{location.lower()}_tmy_plots{suffix}.png', 
                bbox_inches='tight', dpi=300)
    plt.close()


def generate_eda_report(df, location, output_dir="reports", df_final=None):
    """
    Genera un informe EDA (Análisis Exploratorio de Datos) con:
    - Estadísticas de valores faltantes y outliers
    - Estadísticas descriptivas
    - Recomendaciones de limpieza
    - (Opcional) Resumen numérico de la limpieza final si se pasa df_final
    El informe se guarda en 'reports/{localidad}_eda_report.txt'.
    """
    # Crear directorio para informes si no existe
    Path(output_dir).mkdir(exist_ok=True)

    # Crear archivo de informe
    report_path = Path(output_dir) / f"{location.lower()}_eda_report.txt"
    print(f"[DEBUG] Generando reporte EDA en: {report_path}")

    # Definir límites de outliers
    outlier_limits = {
        'GHI': 1200,  # Actualizado para coincidir con limpiar_TMY_completo
        'DNI': 1300,
        'DHI': 600
    }

    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(f"=== Informe EDA - {location} ===\n\n")

        # 1. Información general
        f.write("1. INFORMACIÓN GENERAL\n")
        f.write("-" * 50 + "\n")
        f.write(f"Número total de registros: {len(df)}\n")
        f.write(f"Período: {df['Month'].min()}/{df['Day'].min()} - {df['Month'].max()}/{df['Day'].max()}\n\n")

        # 2. Análisis de valores faltantes
        f.write("2. ANÁLISIS DE VALORES FALTANTES\n")
        f.write("-" * 50 + "\n")
        nan_counts = df[['GHI', 'DNI', 'DHI']].isna().sum()
        nan_percentages = (nan_counts / len(df)) * 100

        for col in ['GHI', 'DNI', 'DHI']:
            f.write(f"{col}:\n")
            f.write(f"  - Número de valores faltantes: {nan_counts[col]}\n")
            f.write(f"  - Porcentaje de valores faltantes: {nan_percentages[col]:.2f}%\n")

            # Análisis de secuencias de NaN
            if nan_counts[col] > 0:
                nan_sequences = df[col].isna().astype(int).groupby(
                    (df[col].isna().astype(int).diff() != 0).cumsum()
                ).cumsum()
                max_consecutive = nan_sequences.max()
                f.write(f"  - Máxima secuencia de NaN consecutivos: {max_consecutive}\n")
        f.write("\n")

        # 3. Análisis de Outliers
        f.write("3. ANÁLISIS DE OUTLIERS\n")
        f.write("-" * 50 + "\n")
        f.write("Criterios de outliers:\n")
        f.write("- GHI > 1200 W/m² o < 0\n")  # Actualizado
        f.write("- DNI > 1300 W/m² o < 0\n")
        f.write("- DHI > 600 W/m² o < 0\n\n")

        for col in ['GHI', 'DNI', 'DHI']:
            # Identificar outliers (valores negativos o mayores al límite)
            neg_outliers = df[df[col] < 0][col]
            high_outliers = df[df[col] > outlier_limits[col]][col]
            total_outliers = len(neg_outliers) + len(high_outliers)

            f.write(f"{col}:\n")
            f.write(f"  - Número total de outliers: {total_outliers}\n")
            f.write(f"  - Porcentaje de outliers: {(total_outliers/len(df))*100:.2f}%\n")

            if len(neg_outliers) > 0:
                f.write(f"  - Outliers negativos: {len(neg_outliers)} ({len(neg_outliers)/total_outliers*100:.2f}% del total de outliers)\n")
                f.write(f"    * Valor mínimo: {neg_outliers.min():.2f} W/m²\n")
                f.write(f"    * Valor máximo negativo: {neg_outliers.max():.2f} W/m²\n")
                f.write(f"    * Media de outliers negativos: {neg_outliers.mean():.2f} W/m²\n")

            if len(high_outliers) > 0:
                f.write(f"  - Outliers altos: {len(high_outliers)} ({len(high_outliers)/total_outliers*100:.2f}% del total de outliers)\n")
                f.write(f"    * Valor mínimo: {high_outliers.min():.2f} W/m²\n")
                f.write(f"    * Valor máximo: {high_outliers.max():.2f} W/m²\n")
                f.write(f"    * Media de outliers altos: {high_outliers.mean():.2f} W/m²\n")

            # Análisis temporal de outliers
            if total_outliers > 0:
                f.write("  - Distribución temporal de outliers:\n")
                for month in range(1, 13):
                    month_outliers = len(df[(df['Month'] == month) & 
                                          ((df[col] < 0) | (df[col] > outlier_limits[col]))])
                    if month_outliers > 0:
                        f.write(f"    * Mes {month}: {month_outliers} outliers\n")
        f.write("\n")
        
        # 4. Estadísticas descriptivas (excluyendo outliers)
        f.write("4. ESTADÍSTICAS DESCRIPTIVAS (EXCLUYENDO OUTLIERS)\n")
        f.write("-" * 50 + "\n")
        for col in ['GHI', 'DNI', 'DHI']:
            f.write(f"{col}:\n")
            # Filtrar valores válidos (no outliers)
            valid_data = df[(df[col] >= 0) & (df[col] <= outlier_limits[col])][col]
            stats = valid_data.describe()
            f.write(f"  - Media: {stats['mean']:.2f} W/m²\n")
            f.write(f"  - Desviación estándar: {stats['std']:.2f} W/m²\n")
            f.write(f"  - Mínimo: {stats['min']:.2f} W/m²\n")
            f.write(f"  - Máximo: {stats['max']:.2f} W/m²\n")
            f.write(f"  - Mediana: {stats['50%']:.2f} W/m²\n")
            f.write(f"  - Q1 (25%): {stats['25%']:.2f} W/m²\n")
            f.write(f"  - Q3 (75%): {stats['75%']:.2f} W/m²\n")
        f.write("\n")

        # 5. Recomendaciones
        f.write("5. RECOMENDACIONES\n")
        f.write("-" * 50 + "\n")
        for col in ['GHI', 'DNI', 'DHI']:
            f.write(f"{col}:\n")
            if nan_counts[col] > 0:
                f.write(f"  - Considerar interpolación para {nan_counts[col]} valores faltantes ")
                if 'max_consecutive' in locals() and max_consecutive > 4:
                    f.write(f"(¡Alerta! Hay secuencias de hasta {max_consecutive} NaN consecutivos)\n")
                else:
                    f.write("(secuencias cortas, adecuadas para interpolación)\n")

            neg_count = len(df[df[col] < 0])
            high_count = len(df[df[col] > outlier_limits[col]])
            if neg_count > 0 or high_count > 0:
                f.write(f"  - Reemplazar {neg_count + high_count} outliers:\n")
                if neg_count > 0:
                    f.write(f"    * {neg_count} valores negativos con 0\n")
                if high_count > 0:
                    f.write(f"    * {high_count} valores > {outlier_limits[col]} W/m² con {outlier_limits[col]} W/m²\n")
                f.write(f"  - Revisar la calidad de los datos en los meses con mayor concentración de outliers\n")

        f.write("\nRecomendaciones generales:\n")
        f.write("1. Reemplazar todos los valores negativos con 0\n")
        f.write("2. Limitar los valores máximos a los umbrales físicos:\n")
        f.write("   - GHI: 1200 W/m²\n")  # Actualizado
        f.write("   - DNI: 1300 W/m²\n")
        f.write("   - DHI: 600 W/m²\n")
        f.write("3. Considerar la interpolación solo para secuencias cortas de NaN (≤ 4 horas)\n")
        f.write("4. Revisar la calidad de los datos en los meses con mayor concentración de outliers\n")
        f.write("5. Documentar el proceso de limpieza y las decisiones tomadas para el manejo de outliers\n")
        # --- SECCIÓN DE RESULTADOS FINALES ---
        if df_final is not None:
            print(f"[DEBUG] Escribiendo resultados finales para {location}...")
            f.write("\n=== RESULTADOS FINALES DE LA LIMPIEZA ===\n")
            outlier_limits = {'GHI': 1200, 'DNI': 1300, 'DHI': 600}
            for col in ['GHI', 'DNI', 'DHI']:
                neg_outliers = len(df_final[df_final[col] < 0])
                high_outliers = len(df_final[df_final[col] > outlier_limits[col]])
                total_outliers = neg_outliers + high_outliers
                nans = df_final[col].isna().sum()
                f.write(f"{col}: {total_outliers} outliers (Negativos: {neg_outliers}, Altos: {high_outliers}), {nans} NaNs\n")

def analisis_estacional(df, location):
    """
    Realiza un análisis estacional de los datos solares para una localidad.
    Imprime estadísticas por estación (verano, otoño, invierno, primavera).
    """
    print(f"\n=== ANÁLISIS ESTACIONAL - {location} ===")
    print("=" * 50)

    # Definir estaciones
    estaciones = {
        'Verano': [12, 1, 2],
        'Otoño': [3, 4, 5],
        'Invierno': [6, 7, 8],
        'Primavera': [9, 10, 11]
    }

    # Crear DataFrame para almacenar estadísticas estacionales
    stats_estacionales = pd.DataFrame(index=estaciones.keys(), 
                                    columns=['GHI Promedio', 'DNI Promedio', 'DHI Promedio',
                                            'GHI Máximo', 'DNI Máximo', 'DHI Máximo',
                                            'Horas de Sol', 'Energía Total'])

    # Calcular estadísticas por estación
    for estacion, meses in estaciones.items():
        df_estacion = df[df['Month'].isin(meses)]

        # Estadísticas básicas
        stats_estacionales.loc[estacion, 'GHI Promedio'] = df_estacion['GHI'].mean()
        stats_estacionales.loc[estacion, 'DNI Promedio'] = df_estacion['DNI'].mean()
        stats_estacionales.loc[estacion, 'DHI Promedio'] = df_estacion['DHI'].mean()

        stats_estacionales.loc[estacion, 'GHI Máximo'] = df_estacion['GHI'].max()
        stats_estacionales.loc[estacion, 'DNI Máximo'] = df_estacion['DNI'].max()
        stats_estacionales.loc[estacion, 'DHI Máximo'] = df_estacion['DHI'].max()

        # Horas de sol y energía
        stats_estacionales.loc[estacion, 'Horas de Sol'] = len(df_estacion[df_estacion['GHI'] > 0])
        stats_estacionales.loc[estacion, 'Energía Total'] = df_estacion['GHI'].sum() / 1000

    # Mostrar estadísticas estacionales
    print("\nEstadísticas por Estación:")
    print(stats_estacionales.round(2))


def marcar_outliers_nan(df):
    """
    Marca como NaN los valores físicamente inválidos en GHI, DNI y DHI.
    - GHI < 0 o GHI > 1400
    - DNI < 0 o DNI > 1300
    - DHI < 0 o DHI > 600
    Devuelve un DataFrame con los valores inválidos como NaN.
    """
    df = df.copy()
    df['GHI'] = df['GHI'].mask((df['GHI'] < 0) | (df['GHI'] > 1400))
    df['DNI'] = df['DNI'].mask((df['DNI'] < 0) | (df['DNI'] > 1300))
    df['DHI'] = df['DHI'].mask((df['DHI'] < 0) | (df['DHI'] > 600))
    return df


def limpiar_TMY_completo(archivo_entrada, archivo_salida, max_ghi=1400, max_dni=1300, max_dhi=600, interp_limit=6, location=None):
    """
    Limpia y valida físicamente los datos TMY artificiales:
    - Aplica límites físicos y de temporada a GHI, DNI, DHI
    - Interpola NaNs de forma robusta
    - Valida DHI con posición solar
    - Limpia Tdry, Tdew, RH y Pres
    - Guarda el archivo limpio con metadatos
    """
    # Leer metadatos
    with open(archivo_entrada, 'r', encoding='utf-8') as f:
        metadatos = [next(f) for _ in range(2)]
    # Leer datos desde la tercera línea
    df = pd.read_csv(archivo_entrada, skiprows=2)
    # Crear columna datetime y usar como índice
    df["datetime"] = pd.to_datetime(df[["Year", "Month", "Day", "Hour", "Minute"]])
    df = df.set_index("datetime")

    # Ajustar límites según la ubicación
    if location == "Vallenar":
        max_ghi = 1200
        ghi_high_season = 1150
    else:
        max_ghi = 1400
        ghi_high_season = 1250

    # Limpiar GHI con límites dependientes del mes y ubicación
    df["GHI"] = df["GHI"].mask(
        ((df["Month"].between(3, 10)) & (df["GHI"] >= ghi_high_season)) |  # Límite para meses de alta radiación
        ((~df["Month"].between(3, 10)) & (df["GHI"] > max_ghi)) |  # Límite general
        (df["GHI"] < 0)
    )
    df["GHI"] = df["GHI"].interpolate(method='linear', limit=interp_limit, limit_direction='both')

    # Limpiar DNI
    df["DNI"] = df["DNI"].mask((df["DNI"] < 0) | (df["DNI"] > max_dni))
    df["DNI"] = df["DNI"].interpolate(method='linear', limit=interp_limit, limit_direction='both')

    # Limpiar DHI con chequeos físicos avanzados
    # Calcular posición solar para validación física
    lat = -26.2533 if location == "Salvador" else -22.4661 if location == "Calama" else -28.5766
    lon = -69.0522 if location == "Salvador" else -68.9244 if location == "Calama" else -70.7601
    solar_position = get_solarposition(df.index, latitude=lat, longitude=lon)
    cos_zenith = np.cos(np.radians(solar_position["zenith"]))
    dhi_est = (df["GHI"] - df["DNI"] * cos_zenith).clip(lower=0)

    cond_invalid_dhi = (
        (df["DHI"] < 0) |
        (df["DHI"] > max_dhi) |
        (df["DHI"] > df["GHI"]) |
        (df["DHI"] > dhi_est + 30) |
        (df["DHI"] > 0.95 * df["GHI"]) |
        ((solar_position["zenith"] > 90) & (df["DHI"] > 5))
    )
    df.loc[cond_invalid_dhi, "DHI"] = np.nan

    # Interpolación robusta para DHI
    def interpolar_robusto(serie, limit):
        nan_groups = serie.isna().astype(int).groupby(serie.notna().astype(int).cumsum()).sum()
        if (nan_groups > limit).any():
            mask = serie.isna()
            for idx, size in nan_groups[nan_groups > limit].items():
                mask[mask.groupby(mask.cumsum()).ngroup() == idx] = False
            serie_interp = serie.interpolate(method='linear', limit=limit, limit_direction='both')
            serie[mask] = serie_interp[mask]
            return serie
        else:
            return serie.interpolate(method='linear', limit=limit, limit_direction='both')

    df["DHI"] = interpolar_robusto(df["DHI"], interp_limit)

    # Limpiar Tdry, Tdew, RH y Pres
    # Definir límites más realistas para cada parámetro
    temp_limits = {
        'Tdry': (-10, 50),  # Limitar entre -10°C y 50°C
        'Tdew': (-10, 50),  # Limitar entre -10°C y 50°C
        'RH': (1, 100),     # Forzar al rango físico [1%, 100%]
        'Pres': (760, 790)  # Limitar entre 760 y 790 hPa
    }

    for param, (min_val, max_val) in temp_limits.items():
        df[param] = df[param].mask((df[param] < min_val) | (df[param] > max_val))
        df[param] = df[param].interpolate(method='linear', limit=interp_limit, limit_direction='both')

    # Interpolación para corregir NaNs
    for col in ['GHI', 'DNI', 'DHI', 'Tdry', 'Tdew', 'RH', 'Pres']:
        df[col] = df[col].interpolate(method='linear', limit_direction='both')

    # Opcional: Rellenar cualquier NaN restante con la media de la columna
    df.fillna(df.mean(), inplace=True)

    # Restaurar columnas separadas
    df = df.reset_index()
    df["Year"] = df["datetime"].dt.year
    df["Month"] = df["datetime"].dt.month
    df["Day"] = df["datetime"].dt.day
    df["Hour"] = df["datetime"].dt.hour
    df["Minute"] = df["datetime"].dt.minute

    # Reordenar columnas
    columnas_fecha = ["Year", "Month", "Day", "Hour", "Minute"]
    columnas_finales = columnas_fecha + [col for col in df.columns if col not in columnas_fecha + ["datetime"]]
    df = df[columnas_finales]

    # --- NUEVOS OUTLIERS ESPECÍFICOS POR UBICACIÓN Y MES ---
    if location == "Vallenar":
        # GHI > 1000 entre abril y agosto
        df["GHI"] = df["GHI"].mask((df["Month"].between(4, 8)) & (df["GHI"] > 1000))
        # DNI > 1150 todo el año
        df["DNI"] = df["DNI"].mask(df["DNI"] > 1150)
    elif location == "Calama":
        # GHI > 1000 entre mayo y julio
        df["GHI"] = df["GHI"].mask((df["Month"].between(5, 7)) & (df["GHI"] > 1000))
        # DNI > 1200 todo el año
        df["DNI"] = df["DNI"].mask(df["DNI"] > 1200)
    elif location == "Salvador":
        # GHI > 1100 entre abril y agosto
        df["GHI"] = df["GHI"].mask((df["Month"].between(4, 8)) & (df["GHI"] > 1100))
    # --- FIN NUEVOS OUTLIERS ---

    # Después de limpiar los datos
    df_limpio = df  # Asegúrate de que df_limpio esté definido
    revisar_outliers_final(df_limpio, location)

    # Guardar nuevo archivo con metadatos originales
    with open(archivo_salida, 'w', encoding='utf-8') as f:
        f.writelines(metadatos)
        f.write(",".join(df.columns) + "\n")
        df.to_csv(f, index=False, header=False)
    print(f"✅ Archivo limpio guardado en: {archivo_salida}")


def revisar_outliers_final(df, location):
    """
    Imprime en consola el resumen de outliers y NaNs tras la limpieza final.
    """
    print(f"\n=== REVISIÓN FINAL DE OUTLIERS Y NaNs - {location} ===")
    outlier_limits = {'GHI': 1200, 'DNI': 1300, 'DHI': 600}  # Límites de outliers

    for col in ['GHI', 'DNI', 'DHI']:
        neg_outliers = len(df[df[col] < 0])
        high_outliers = len(df[df[col] > outlier_limits[col]])
        total_outliers = neg_outliers + high_outliers
        nans = df[col].isna().sum()
        print(f"{col}: {total_outliers} outliers, {nans} NaNs")
        if total_outliers > 0:
            print(f"  - Negativos: {neg_outliers}")
            print(f"  - Altos: {high_outliers}")
        if nans > 0:
            print(f"  - NaNs: {nans}")


def generar_resumen_comparativo(dfs, locations):
    """
    Imprime en consola un resumen comparativo de NaNs y outliers para todas las localidades.
    """
    print("\n=== RESUMEN COMPARATIVO DE DATOS SOLARES ===")
    print("=" * 50)

    # Análisis de valores faltantes
    print("\nAnálisis de Valores Faltantes:")
    for df, location in zip(dfs, locations):
        nan_counts = df[['GHI', 'DNI', 'DHI']].isna().sum()
        print(f"\n{location}:")
        for col in ['GHI', 'DNI', 'DHI']:
            print(f"  {col}: {nan_counts[col]} valores faltantes ({(nan_counts[col]/len(df))*100:.2f}%)")

    # Análisis de outliers
    print("\nAnálisis de Outliers:")
    outlier_limits = {'GHI': 1200, 'DNI': 1300, 'DHI': 400}  # Actualizado
    for df, location in zip(dfs, locations):
        print(f"\n{location}:")
        for col in ['GHI', 'DNI', 'DHI']:
            neg_outliers = len(df[df[col] < 0])
            high_outliers = len(df[df[col] > outlier_limits[col]])
            total_outliers = neg_outliers + high_outliers
            print(f"  {col}: {total_outliers} outliers ({(total_outliers/len(df))*100:.2f}%)")
            if total_outliers > 0:
                print(f"    - Negativos: {neg_outliers}")
                print(f"    - Altos: {high_outliers}")


def reconstruir_TMY_con_fechas_originales(archivo_corrupto, archivo_limpio, archivo_salida):
    """
    Reconstruye un archivo TMY limpio usando las fechas originales del archivo fuente.
    Mantiene los metadatos y el formato TMY.
    """
    # Leer datos limpios
    df_limpio = pd.read_csv(archivo_limpio, skiprows=2)
    # Leer datos originales (sin metadatos)
    df_original = pd.read_csv(archivo_corrupto)
    # Reemplazar columnas de fecha por las originales
    for col in ['Year', 'Month', 'Day', 'Hour', 'Minute']:
        if col in df_original.columns and col in df_limpio.columns:
            df_limpio[col] = df_original[col].values
    # Reordenar columnas para mantener el formato
    columnas_fecha = ['Year', 'Month', 'Day', 'Hour', 'Minute']
    columnas_finales = columnas_fecha + [col for col in df_limpio.columns if col not in columnas_fecha]
    df_limpio = df_limpio[columnas_finales]
    # Leer metadatos (primeras 3 líneas del archivo limpio)
    with open(archivo_limpio, 'r', encoding='utf-8') as f:
        metadatos = [next(f) for _ in range(3)]
    # Guardar el nuevo archivo
    with open(archivo_salida, 'w', encoding='utf-8') as f:
        f.writelines(metadatos)
        df_limpio.to_csv(f, index=False, header=False)
    print(f"✅ Archivo TMY limpio con fechas originales guardado en: {archivo_salida}")


def scan_nan_gaps(df):
    """
    Muestra en consola cuántos NaN y los huecos más largos en GHI, DNI, DHI.
    """
    for col in ['GHI', 'DNI', 'DHI']:
        nan_count = df[col].isna().sum()
        if nan_count > 0:
            nan_sequences = df[col].isna().astype(int).groupby(
                (df[col].isna().astype(int).diff() != 0).cumsum()
            ).cumsum()
            max_consecutive = nan_sequences.max()
            print(f"{col}: {nan_count} NaNs, Max gap: {max_consecutive} hours")
        else:
            print(f"{col}: No NaNs")


def fill_nan_hierarchical(df):
    """
    Llena NaNs jerárquicamente:
    1. Interpola huecos ≤ 3 h
    2. Rellena lo restante con el percentil 75 Mes-Hora
    3. Si aún falta, con el percentil 75 anual por Hora
    """
    # Interpolación para huecos pequeños
    df.interpolate(method='linear', limit=3, limit_direction='both', inplace=True)

    # Rellenar con percentil 75 Mes-Hora
    for col in ['GHI', 'DNI', 'DHI']:
        if df[col].isna().sum() > 0:
            df[col] = df.groupby(['Month', 'Hour'])[col].transform(lambda x: x.fillna(x.quantile(0.75)))

    # Rellenar con percentil 75 anual por Hora
    for col in ['GHI', 'DNI', 'DHI']:
        if df[col].isna().sum() > 0:
            df[col] = df.groupby('Hour')[col].transform(lambda x: x.fillna(x.quantile(0.75)))

    return df


def process_and_save_final_tmy(file_path, output_path):
    """
    Aplica el llenado jerárquico de NaNs y guarda el archivo final TMY.
    """
    # Leer el archivo CSV
    with open(file_path, 'r', encoding='utf-8') as f:
        metadatos = [next(f) for _ in range(2)]  # Leer las dos primeras líneas de metadatos
    df = pd.read_csv(file_path, skiprows=2)

    # Poner la columna datetime como índice
    df['datetime'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour', 'Minute']])
    df.set_index('datetime', inplace=True)

    # Escanear huecos de NaN
    print(f"\nEscaneando huecos de NaN en {file_path} antes de llenar:")
    scan_nan_gaps(df)

    # Llenar NaN jerárquicamente
    df = fill_nan_hierarchical(df)

    # Escanear nuevamente para confirmar que no quedan NaN
    print(f"\nEscaneando huecos de NaN en {file_path} después de llenar:")
    scan_nan_gaps(df)

    # Guardar el resultado con el mismo formato TMY
    with open(output_path, 'w', encoding='utf-8') as f:
        f.writelines(metadatos)
        f.write(','.join(df.columns) + '\n')  # Escribir encabezado de columnas
        df.to_csv(f, index=False, header=False)
    print(f"✅ Archivo final guardado en: {output_path}")

# =============================================================
# FLUJO PRINCIPAL DE EJECUCIÓN
# =============================================================

def main_tmy():
    """
    Ejecuta el flujo completo de limpieza, análisis y generación de reportes para las tres localidades.
    - Transforma y limpia los archivos originales
    - Genera gráficos y reportes EDA
    - Reconstruye archivos con fechas originales
    - Llena NaNs y genera archivos finales
    - Genera informes EDA finales con resumen de limpieza
    """
    print("Iniciando procesamiento...")
    # Definir los metadatos para cada sitio
    metadatos_salvador = {
        "Source": "ExpSolar", "Location ID": "00001", "City": "Salvador", "State": "Atacama",
        "Country": "Chile", "Latitude": -26.2533, "Longitude": -69.0522, "Time Zone": -4, "Elevation": 2280
    }
    metadatos_calama = {
        "Source": "ExpSolar", "Location ID": "00002", "City": "Calama", "State": "Antofagasta",
        "Country": "Chile", "Latitude": -22.4661, "Longitude": -68.9244, "Time Zone": -4, "Elevation": 2260
    }
    metadatos_vallenar = {
        "Source": "ExpSolar", "Location ID": "00003", "City": "Vallenar", "State": "Atacama",
        "Country": "Chile", "Latitude": -28.5766, "Longitude": -70.7601, "Time Zone": -4, "Elevation": 441
    }
    # Rutas de archivos
    path_salvador = Path("salvador_corrupted.csv")
    path_calama = Path("calama_corrupted.csv")
    path_vallenar = Path("Vallenar_corrupted.csv")
    # Salidas
    output_salvador = Path("salvador_TMY_artificial.csv")
    output_calama = Path("calama_TMY_artificial.csv")
    output_vallenar = Path("vallenar_TMY_artificial.csv")
    output_salvador_limpio = Path(DATOS_LIMPIOS_DIR / "salvador_TMY_limpio.csv")
    output_calama_limpio = Path(DATOS_LIMPIOS_DIR / "calama_TMY_limpio.csv")
    output_vallenar_limpio = Path(DATOS_LIMPIOS_DIR / "vallenar_TMY_limpio.csv")

    # Procesar cada archivo (sin generar el informe EDA aquí)
    for location, (input_path, output_path, output_limpio, metadata) in [
        ("Salvador", (path_salvador, output_salvador, output_salvador_limpio, metadatos_salvador)),
        ("Calama", (path_calama, output_calama, output_calama_limpio, metadatos_calama)),
        ("Vallenar", (path_vallenar, output_vallenar, output_vallenar_limpio, metadatos_vallenar))
    ]:
        try:
            print(f"\nProcesando {location}...")
            print(f"Leyendo archivo: {input_path}")
            df = transformar_a_tmy_con_metadatos(input_path, output_path, metadata)
            print(f"Transformación completada. Limpiando datos...")
            limpiar_TMY_completo(str(output_path), str(output_limpio), location=location)
            print(f"Limpieza completada. Eliminando archivos temporales...")
            for suffix in ["_GHI_limpio.csv", "_DNI_limpio.csv", "_DHI_limpio.csv"]:
                try:
                    os.remove(str(output_path).replace("_TMY_artificial.csv", suffix))
                except FileNotFoundError:
                    pass
            print(f"Leyendo archivo limpio...")
            df_limpio = pd.read_csv(output_limpio, skiprows=2)
            print(f"Generando gráficos...")
            plot_tmy_data(df_limpio, location)
            print(f"Realizando análisis estacional...")
            analisis_estacional(df_limpio, location)
            print(f"Eliminando archivo temporal...")
            try:
                os.remove(str(output_path))
            except FileNotFoundError:
                pass
            print(f"Procesamiento de {location} completado.")
        except Exception as e:
            print(f"Error procesando {location}: {str(e)}")
            import traceback
            print(traceback.format_exc())

    # Generar resumen comparativo
    dfs_limpios = [
        pd.read_csv(DATOS_LIMPIOS_DIR / "salvador_TMY_limpio.csv", skiprows=2),
        pd.read_csv(DATOS_LIMPIOS_DIR / "calama_TMY_limpio.csv", skiprows=2),
        pd.read_csv(DATOS_LIMPIOS_DIR / "vallenar_TMY_limpio.csv", skiprows=2)
    ]
    locations = ["Salvador", "Calama", "Vallenar"]
    if len(dfs_limpios) == 3:
        print("\nGenerando resumen comparativo...")
        generar_resumen_comparativo(dfs_limpios, locations)
        print("\nReconstruyendo archivos TMY limpios con fechas originales...")
        reconstruir_TMY_con_fechas_originales("salvador_corrupted.csv", DATOS_LIMPIOS_DIR / "salvador_TMY_limpio.csv", DATOS_LIMPIOS_DIR / "salvador_TMY_limpio_originales.csv")
        reconstruir_TMY_con_fechas_originales("calama_corrupted.csv", DATOS_LIMPIOS_DIR / "calama_TMY_limpio.csv", DATOS_LIMPIOS_DIR / "calama_TMY_limpio_originales.csv")
        reconstruir_TMY_con_fechas_originales("Vallenar_corrupted.csv", DATOS_LIMPIOS_DIR / "vallenar_TMY_limpio.csv", DATOS_LIMPIOS_DIR / "vallenar_TMY_limpio_originales.csv")
    else:
        print(f"\nNo se pudo generar el resumen comparativo. Se procesaron {len(dfs_limpios)} de 3 archivos.")

    # Procesar cada archivo limpio y guardar el resultado final
    process_and_save_final_tmy(DATOS_LIMPIOS_DIR / 'calama_TMY_limpio_originales.csv', DATOS_LIMPIOS_DIR / 'calama_TMY_final.csv')
    process_and_save_final_tmy(DATOS_LIMPIOS_DIR / 'salvador_TMY_limpio_originales.csv', DATOS_LIMPIOS_DIR / 'salvador_TMY_final.csv')
    process_and_save_final_tmy(DATOS_LIMPIOS_DIR / 'vallenar_TMY_limpio_originales.csv', DATOS_LIMPIOS_DIR / 'vallenar_TMY_final.csv')

    # Ahora sí, generar el informe EDA con ambos DataFrames
    for location in ["Calama", "Salvador", "Vallenar"]:
        df_limpio = pd.read_csv(DATOS_LIMPIOS_DIR / f"{location.lower()}_TMY_limpio.csv", skiprows=2)
        df_final = pd.read_csv(DATOS_LIMPIOS_DIR / f"{location.lower()}_TMY_final.csv", skiprows=2)
        generate_eda_report(df_limpio, location, df_final=df_final)

if __name__ == "__main__":
    main_tmy() 

Iniciando procesamiento...

Procesando Salvador...
Leyendo archivo: salvador_corrupted.csv
Transformación completada. Limpiando datos...


/tmp/ipykernel_9865/4201889616.py:383: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie[mask] = serie_interp[mask]



=== REVISIÓN FINAL DE OUTLIERS Y NaNs - Salvador ===
GHI: 111 outliers, 1 NaNs
  - Negativos: 0
  - Altos: 111
  - NaNs: 1
DNI: 0 outliers, 0 NaNs
DHI: 0 outliers, 0 NaNs
✅ Archivo limpio guardado en: datos_limpios/salvador_TMY_limpio.csv
Limpieza completada. Eliminando archivos temporales...
Leyendo archivo limpio...
Generando gráficos...
Realizando análisis estacional...

=== ANÁLISIS ESTACIONAL - Salvador ===

Estadísticas por Estación:
          GHI Promedio DNI Promedio DHI Promedio GHI Máximo DNI Máximo  \
Verano      389.684931   541.464676    15.965579     1248.3     1222.7   
Otoño       258.563072   406.613972    14.172056     1167.7     1203.6   
Invierno    197.343727   337.040602    11.540444      978.0     1149.7   
Primavera   354.186401   515.705128    13.635943     1246.4     1223.6   

          DHI Máximo Horas de Sol Energía Total  
Verano         518.2         1201     841.71945  
Otoño          535.7         1039      570.6487  
Invierno       164.5          970 

/tmp/ipykernel_9865/4201889616.py:383: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie[mask] = serie_interp[mask]



=== REVISIÓN FINAL DE OUTLIERS Y NaNs - Calama ===
GHI: 49 outliers, 3 NaNs
  - Negativos: 0
  - Altos: 49
  - NaNs: 3
DNI: 0 outliers, 2 NaNs
  - NaNs: 2
DHI: 0 outliers, 0 NaNs
✅ Archivo limpio guardado en: datos_limpios/calama_TMY_limpio.csv
Limpieza completada. Eliminando archivos temporales...
Leyendo archivo limpio...
Generando gráficos...
Realizando análisis estacional...

=== ANÁLISIS ESTACIONAL - Calama ===

Estadísticas por Estación:
          GHI Promedio DNI Promedio DHI Promedio GHI Máximo DNI Máximo  \
Verano       371.25412   484.361458    16.991319     1254.0     1192.9   
Otoño        276.07951    425.86216    14.890082     1156.1     1163.1   
Invierno    223.032465     379.3744    11.958243      985.3     1127.4   
Primavera   351.340911   488.193816    17.601717     1233.8     1196.4   

          DHI Máximo Horas de Sol Energía Total  
Verano         432.1         1196      801.9089  
Otoño          349.8         1060      609.0314  
Invierno       192.0          

/tmp/ipykernel_9865/4201889616.py:383: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie[mask] = serie_interp[mask]



=== REVISIÓN FINAL DE OUTLIERS Y NaNs - Vallenar ===
GHI: 0 outliers, 2 NaNs
  - NaNs: 2
DNI: 0 outliers, 2 NaNs
  - NaNs: 2
DHI: 0 outliers, 0 NaNs
✅ Archivo limpio guardado en: datos_limpios/vallenar_TMY_limpio.csv
Limpieza completada. Eliminando archivos temporales...
Leyendo archivo limpio...
Generando gráficos...
Realizando análisis estacional...

=== ANÁLISIS ESTACIONAL - Vallenar ===

Estadísticas por Estación:
          GHI Promedio DNI Promedio DHI Promedio GHI Máximo DNI Máximo  \
Verano      352.003056   464.676795    21.866458     1152.6     1111.0   
Otoño        217.90401   326.889253    16.282994     1049.1     1074.7   
Invierno    156.989964   266.652447    14.600317      864.3     1036.3   
Primavera   302.854806   393.443402    18.393246     1164.0     1132.5   

          DHI Máximo Horas de Sol Energía Total  
Verano         268.5         1181      760.3266  
Otoño          260.5         1029     480.91415  
Invierno       332.6          961     346.47685  
Primav

Los datos fueron limpiados manteniendo la estructura original de registros horarios. La interpolación aplicada no introduce discontinuidades notables y mejora la continuidad temporal, permitiendo una entrada confiable a las simulaciones fotovoltaicas. Cada archivo fue validado visualmente para confirmar la corrección de valores anómalos, garantizando que los datasets representan condiciones físicas razonables.


## 3. Simulación de planta PV 
Se utilizó el modelo PVWatts v8 de PySAM para simular una planta fotovoltaica de 50 MWDC en cada una de las tres localidades. La configuración base fue:

- Capacidad: 50 MWDC
- Relación DC/AC: 1.2
- Pérdidas totales: 14 %
- Sistema fijo con seguimiento a 1 eje en algunos casos de prueba

Cada simulación se ejecutó para los datos meteorológicos limpios TMY, generando la producción horaria de energía AC.




In [10]:
"""
Simulación de plantas fotovoltaicas
------------------------------------------------
Este script permite calcular la energía incidente, simular la producción de energía de una planta fotovoltaica,
calcular el LCOE y VAN, y realizar análisis de sensibilidad para distintos parámetros económicos y técnicos.

Requiere archivos TMY de recurso solar para cada localidad y genera resultados y gráficos en la carpeta 'resultados_pv'.

Autor: Nicole Torres
Fecha: 14/05/2025
"""
# =============================
# 🔧 Definición de funciones
# =============================
def calculate_incident_energy(solar_resource_file):
    """
    Calcula la energía incidente anual a partir de datos TMY.
    Args:
        solar_resource_file (str): Ruta al archivo de recurso solar (TMY)
    Returns:
        float: Energía incidente anual en kWh/m²
    """
    try:
        df = pd.read_csv(solar_resource_file, skiprows=2, sep=',', on_bad_lines='skip', encoding='utf-8')
        ghi_columns = [col for col in df.columns if 'GHI' in col.upper()]
        if not ghi_columns:
            raise ValueError(f"No se encontró columna GHI en {solar_resource_file}")
        ghi_column = ghi_columns[0]
        annual_incident_energy = df[ghi_column].sum() / 1000  # W/m² a kWh/m²
        return annual_incident_energy
    except Exception as e:
        print(f"Error al leer el archivo {solar_resource_file}: {str(e)}")
        return None

def calculate_lcoe(annual_energy, system_capacity_kw, fixed_charge_rate=0.08, project_lifetime=25,
                  capex_pv=800, fixed_om_cost=50, variable_om_cost=0.01, inverter_lifetime=10,
                  system_losses=14.0):
    """
    Calcula el Costo Nivelado de Energía (LCOE).
    Args:
        annual_energy (float): Producción anual de energía en kWh
        system_capacity_kw (float): Capacidad del sistema en kW
        fixed_charge_rate (float): Tasa de cargo fijo (por defecto 8%)
        project_lifetime (int): Vida útil del proyecto en años (por defecto 25)
        capex_pv (float): Costo de capital por kW
        fixed_om_cost (float): Costo fijo de O&M por kW/año
        variable_om_cost (float): Costo variable de O&M por kWh
        inverter_lifetime (int): Vida útil del inversor en años
        system_losses (float): Pérdidas del sistema en %
    Returns:
        float: LCOE en $/kWh
    """
    capital_cost = system_capacity_kw * capex_pv
    num_replacements = int(project_lifetime / inverter_lifetime) - 1
    if num_replacements > 0:
        inverter_cost = system_capacity_kw * 200  # $200/kW para reemplazo
        for i in range(num_replacements):
            replacement_year = (i + 1) * inverter_lifetime
            capital_cost += inverter_cost / (1 + fixed_charge_rate)**replacement_year
    fixed_operating_cost = system_capacity_kw * fixed_om_cost
    variable_operating_cost = variable_om_cost
    annual_energy = annual_energy * (1 - system_losses/100)
    discount_rate = fixed_charge_rate
    pv_factor = (1 - (1 + discount_rate)**-project_lifetime) / discount_rate
    total_capital_cost = capital_cost
    total_fixed_om_cost = fixed_operating_cost * pv_factor
    total_variable_om_cost = variable_operating_cost * annual_energy * pv_factor
    total_energy = annual_energy * project_lifetime
    lcoe = (total_capital_cost + total_fixed_om_cost + total_variable_om_cost) / total_energy
    return lcoe

def calculate_npv(
    annual_energy, 
    system_capacity_kw, 
    spot_price=0.06,  # Precio de venta de energía ($/kWh)
    fixed_charge_rate=0.08, 
    project_lifetime=25,
    capex_pv=800, 
    fixed_om_cost=50, 
    variable_om_cost=0.01, 
    inverter_lifetime=10,
    system_losses=14.0
):
    """
    Calcula el Valor Actual Neto (VAN/NPV) del proyecto.
    Args:
        annual_energy (float): Producción anual de energía en kWh
        system_capacity_kw (float): Capacidad del sistema en kW
        spot_price (float): Precio de venta de energía ($/kWh)
        fixed_charge_rate (float): Tasa de descuento
        project_lifetime (int): Vida útil del proyecto en años
        capex_pv (float): Costo de capital por kW
        fixed_om_cost (float): Costo fijo de O&M por kW/año
        variable_om_cost (float): Costo variable de O&M por kWh
        inverter_lifetime (int): Vida útil del inversor en años
        system_losses (float): Pérdidas del sistema en %
    Returns:
        float: VAN en dólares
    """
    capital_cost = system_capacity_kw * capex_pv
    npv = -capital_cost
    annual_energy = annual_energy * (1 - system_losses / 100)
    for year in range(1, project_lifetime + 1):
        revenue = annual_energy * spot_price
        fixed_om = system_capacity_kw * fixed_om_cost
        variable_om = variable_om_cost * annual_energy
        inverter_replacement = 0
        if inverter_lifetime > 0 and year % inverter_lifetime == 0 and year != project_lifetime:
            inverter_replacement = system_capacity_kw * 200
        cash_flow = revenue - fixed_om - variable_om - inverter_replacement
        npv += cash_flow / (1 + fixed_charge_rate) ** year
    return npv

def run_sensitivity_analysis(annual_energy, system_capacity_kw, base_lcoe, location_name):
    """
    Realiza análisis de sensibilidad para el LCOE respecto a distintos parámetros.
    Args:
        annual_energy (float): Producción anual base
        system_capacity_kw (float): Capacidad del sistema
        base_lcoe (float): LCOE base
        location_name (str): Nombre de la localidad
    Returns:
        tuple: (parameters, variations, impacts)
    """
    parameters = ['FCR', 'CapEx PV', 'Spot Price', 'Inverter Lifetime', 'System Losses']
    base_params = {
        'FCR': 0.08,
        'CapEx PV': 1000,
        'Spot Price': 0.06,
        'Inverter Lifetime': 10,
        'System Losses': 14.0
    }
    variations = {
        'FCR': {'base': base_params['FCR'], 'low': 0.06, 'high': 0.10},
        'CapEx PV': {'base': base_params['CapEx PV'], 'low': 800, 'high': 1200},
        'Spot Price': {'base': base_params['Spot Price'], 'low': 0.005, 'high': 0.015},
        'Inverter Lifetime': {'base': base_params['Inverter Lifetime'], 'low': 8, 'high': 12},
        'System Losses': {'base': base_params['System Losses'], 'low': 12.0, 'high': 16.0}
    }
    impacts = []
    print(f"\nAnálisis de sensibilidad para {location_name}:")
    print(f"Producción anual base: {annual_energy/1e6:.2f} GWh")
    print(f"LCOE base: {base_lcoe:.4f} $/kWh")
    for param in parameters:
        # Variación baja
        if param == 'FCR':
            lcoe_low = calculate_lcoe(annual_energy, system_capacity_kw, fixed_charge_rate=variations[param]['low'])
        elif param == 'CapEx PV':
            lcoe_low = calculate_lcoe(annual_energy, system_capacity_kw, capex_pv=variations[param]['low'])
        elif param == 'Spot Price':
            lcoe_low = calculate_lcoe(annual_energy, system_capacity_kw, variable_om_cost=variations[param]['low'])
        elif param == 'Inverter Lifetime':
            lcoe_low = calculate_lcoe(annual_energy, system_capacity_kw, inverter_lifetime=variations[param]['low'])
        else:
            lcoe_low = calculate_lcoe(annual_energy, system_capacity_kw, system_losses=variations[param]['low'])
        # Variación alta
        if param == 'FCR':
            lcoe_high = calculate_lcoe(annual_energy, system_capacity_kw, fixed_charge_rate=variations[param]['high'])
        elif param == 'CapEx PV':
            lcoe_high = calculate_lcoe(annual_energy, system_capacity_kw, capex_pv=variations[param]['high'])
        elif param == 'Spot Price':
            lcoe_high = calculate_lcoe(annual_energy, system_capacity_kw, variable_om_cost=variations[param]['high'])
        elif param == 'Inverter Lifetime':
            lcoe_high = calculate_lcoe(annual_energy, system_capacity_kw, inverter_lifetime=variations[param]['high'])
        else:
            lcoe_high = calculate_lcoe(annual_energy, system_capacity_kw, system_losses=variations[param]['high'])
        impact_low = (lcoe_low - base_lcoe) / base_lcoe * 100
        impact_high = (lcoe_high - base_lcoe) / base_lcoe * 100
        impacts.append((impact_low, impact_high))
        print(f"\n{param}:")
        print(f"  Variación baja: {variations[param]['low']} -> LCOE: {lcoe_low:.4f} $/kWh (Impacto: {impact_low:.1f}%)")
        print(f"  Variación alta: {variations[param]['high']} -> LCOE: {lcoe_high:.4f} $/kWh (Impacto: {impact_high:.1f}%)")
    return parameters, variations, impacts

def run_npv_sensitivity_analysis(
    annual_energy, 
    system_capacity_kw, 
    base_npv, 
    location_name,
    spot_price=0.12
):
    """
    Realiza análisis de sensibilidad sobre el VAN (NPV) para distintos parámetros.
    Args:
        annual_energy (float): Producción anual base
        system_capacity_kw (float): Capacidad del sistema
        base_npv (float): VAN base
        location_name (str): Nombre de la localidad
        spot_price (float): Precio de venta de energía
    Returns:
        tuple: (parameters, variations, impacts)
    """
    parameters = ['FCR', 'CapEx PV', 'Spot Price', 'Inverter Lifetime', 'System Losses']
    base_params = {
        'FCR': 0.08,
        'CapEx PV': 1000,
        'Spot Price': spot_price,
        'Inverter Lifetime': 10,
        'System Losses': 14.0
    }
    variations = {
        'FCR': {'low': 0.06, 'high': 0.10},
        'CapEx PV': {'low': 800, 'high': 1200},
        'Spot Price': {'low': 0.03, 'high': 0.07},
        'Inverter Lifetime': {'low': 8, 'high': 12},
        'System Losses': {'low': 12.0, 'high': 16.0}
    }
    impacts = []
    print(f"\nAnálisis de sensibilidad VAN para {location_name}:")
    print(f"VAN base: ${base_npv:,.2f}")
    for param in parameters:
        # Variación baja
        kwargs = dict(
            annual_energy=annual_energy,
            system_capacity_kw=system_capacity_kw,
            spot_price=base_params['Spot Price'],
            fixed_charge_rate=base_params['FCR'],
            project_lifetime=20,
            capex_pv=base_params['CapEx PV'],
            fixed_om_cost=50,
            variable_om_cost=0.01,
            inverter_lifetime=base_params['Inverter Lifetime'],
            system_losses=base_params['System Losses']
        )
        if param == 'FCR':
            kwargs['fixed_charge_rate'] = variations[param]['low']
        elif param == 'CapEx PV':
            kwargs['capex_pv'] = variations[param]['low']
        elif param == 'Spot Price':
            kwargs['spot_price'] = variations[param]['low']
        elif param == 'Inverter Lifetime':
            kwargs['inverter_lifetime'] = variations[param]['low']
        elif param == 'System Losses':
            kwargs['system_losses'] = variations[param]['low']
        npv_low = calculate_npv(**kwargs)
        # Variación alta
        kwargs = dict(
            annual_energy=annual_energy,
            system_capacity_kw=system_capacity_kw,
            spot_price=base_params['Spot Price'],
            fixed_charge_rate=base_params['FCR'],
            project_lifetime=20,
            capex_pv=base_params['CapEx PV'],
            fixed_om_cost=50,
            variable_om_cost=0.01,
            inverter_lifetime=base_params['Inverter Lifetime'],
            system_losses=base_params['System Losses']
        )
        if param == 'FCR':
            kwargs['fixed_charge_rate'] = variations[param]['high']
        elif param == 'CapEx PV':
            kwargs['capex_pv'] = variations[param]['high']
        elif param == 'Spot Price':
            kwargs['spot_price'] = variations[param]['high']
        elif param == 'Inverter Lifetime':
            kwargs['inverter_lifetime'] = variations[param]['high']
        elif param == 'System Losses':
            kwargs['system_losses'] = variations[param]['high']
        npv_high = calculate_npv(**kwargs)
        impact_low = (npv_low - base_npv) / abs(base_npv) * 100
        impact_high = (npv_high - base_npv) / abs(base_npv) * 100
        impacts.append((impact_low, impact_high))
        print(f"\n{param}:")
        print(f"  Variación baja: {variations[param]['low']} -> VAN: ${npv_low:,.2f} (Impacto: {impact_low:.1f}%)")
        print(f"  Variación alta: {variations[param]['high']} -> VAN: ${npv_high:,.2f} (Impacto: {impact_high:.1f}%)")
    return parameters, variations, impacts

def plot_tornado(parameters, impacts, location_name):
    """
    Genera un gráfico tornado para el análisis de sensibilidad del LCOE.
    Args:
        parameters (list): Lista de nombres de parámetros
        impacts (list): Lista de tuplas (bajo, alto) de impacto
        location_name (str): Nombre de la localidad
    """
    sorted_indices = np.argsort([abs(high - low) for low, high in impacts])
    sorted_params = [parameters[i] for i in sorted_indices]
    sorted_impacts = [impacts[i] for i in sorted_indices]
    fig, ax = plt.subplots(figsize=(10, 6))
    y_pos = np.arange(len(sorted_params))
    width = 0.35
    ax.barh(y_pos - width/2, [low for low, _ in sorted_impacts], width, color='red', label='Variación baja')
    ax.barh(y_pos + width/2, [high for _, high in sorted_impacts], width, color='green', label='Variación alta')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(sorted_params)
    ax.set_xlabel('Impacto en LCOE (%)')
    ax.set_title(f'Análisis de sensibilidad LCOE - {location_name}')
    ax.legend()
    ax.grid(True, axis='x')
    for i, (low, high) in enumerate(sorted_impacts):
        ax.text(low, i - width/2, f'{low:.1f}%', ha='right', va='center')
        ax.text(high, i + width/2, f'{high:.1f}%', ha='left', va='center')
    plt.tight_layout()
    plt.savefig(RESULTADOS_PV_DIR / f'tornado_analysis_{location_name.lower()}.png')
    plt.close()

def plot_npv_tornado(parameters, impacts, location_name):
    """
    Genera un gráfico tornado para el análisis de sensibilidad del VAN.
    Args:
        parameters (list): Lista de nombres de parámetros
        impacts (list): Lista de tuplas (bajo, alto) de impacto
        location_name (str): Nombre de la localidad
    """
    sorted_indices = np.argsort([abs(high - low) for low, high in impacts])
    sorted_params = [parameters[i] for i in sorted_indices]
    sorted_impacts = [impacts[i] for i in sorted_indices]
    fig, ax = plt.subplots(figsize=(10, 6))
    y_pos = np.arange(len(sorted_params))
    width = 0.35
    ax.barh(y_pos - width/2, [low for low, _ in sorted_impacts], width, color='red', label='Variación baja')
    ax.barh(y_pos + width/2, [high for _, high in sorted_impacts], width, color='green', label='Variación alta')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(sorted_params)
    ax.set_xlabel('Impacto en VAN (%)')
    ax.set_title(f'Análisis de sensibilidad VAN - {location_name}')
    ax.legend()
    ax.grid(True, axis='x')
    for i, (low, high) in enumerate(sorted_impacts):
        ax.text(low, i - width/2, f'{low:.1f}%', ha='right', va='center')
        ax.text(high, i + width/2, f'{high:.1f}%', ha='left', va='center')
    plt.tight_layout()
    plt.savefig(RESULTADOS_PV_DIR / f'tornado_npv_{location_name.lower()}.png')
    plt.close()

def simulate_pv_plant(solar_resource_file, system_capacity_kw, location_name):
    """
    Simula una planta FV usando el modelo PVWatts.
    Args:
        solar_resource_file (str): Ruta al archivo de recurso solar
        system_capacity_kw (float): Capacidad del sistema en kW
        location_name (str): Nombre de la localidad
    Returns:
        tuple: (annual_energy, lcoe, incident_energy, hourly_power)
    """
    incident_energy = calculate_incident_energy(solar_resource_file)
    if incident_energy is None:
        print(f"Advertencia: No se pudo calcular la energía incidente para {location_name}")
        incident_energy = 0
    pv_model = pv.new()
    pv_model.SolarResource.solar_resource_file = solar_resource_file
    pv_model.SystemDesign.system_capacity = system_capacity_kw
    pv_model.SystemDesign.dc_ac_ratio = 1.2
    pv_model.SystemDesign.array_type = 1  # Fijo
    pv_model.SystemDesign.azimuth = 180   # Sur
    pv_model.SystemDesign.tilt = 20       # Inclinación 20°
    pv_model.SystemDesign.gcr = 0.4       # Relación cobertura suelo
    pv_model.SystemDesign.inv_eff = 96    # Eficiencia inversor
    pv_model.SystemDesign.losses = 14.0   # Pérdidas
    pv_model.execute()
    annual_energy = pv_model.Outputs.annual_energy 
    hourly_power = np.array(pv_model.Outputs.ac) / 1000
    base_lcoe = calculate_lcoe(annual_energy, system_capacity_kw, fixed_charge_rate=0.08, project_lifetime=20)
    parameters, variations, impacts = run_sensitivity_analysis(annual_energy, system_capacity_kw, base_lcoe, location_name)
    plot_tornado(parameters, impacts, location_name)
    return annual_energy, base_lcoe, incident_energy, hourly_power

def plot_combined_tornado(all_results):
    """
    Genera un gráfico tornado combinado para todas las localidades.
    Args:
        all_results (dict): Resultados de cada localidad
    """
    fig, ax = plt.subplots(figsize=(12, 8))
    parameters = ['FCR', 'CapEx PV', 'Spot Price', 'Inverter Lifetime', 'System Losses']
    y_pos = np.arange(len(parameters))
    width = 0.25
    colors = {
        'Calama': 'red',
        'Salvador': 'green',
        'Vallenar': 'blue'
    }
    for i, location in enumerate(all_results.keys()):
        impacts = all_results[location]['impacts']
        sorted_indices = np.argsort([abs(high - low) for low, high in impacts])
        sorted_impacts = [impacts[i] for i in sorted_indices]
        offset = (i - 1) * width
        low_bars = ax.barh(y_pos + offset, [low for low, _ in sorted_impacts], width, color=colors[location], alpha=0.6, label=f'{location} (Baja)')
        high_bars = ax.barh(y_pos + offset, [high for _, high in sorted_impacts], width, color=colors[location], alpha=0.3, label=f'{location} (Alta)')
        for j, bar in enumerate(low_bars):
            width_bar = bar.get_width()
            ax.text(width_bar, bar.get_y() + bar.get_height()/2, f'{width_bar:.1f}%', ha='right', va='center', fontsize=8)
        for j, bar in enumerate(high_bars):
            width_bar = bar.get_width()
            ax.text(width_bar, bar.get_y() + bar.get_height()/2, f'{width_bar:.1f}%', ha='left', va='center', fontsize=8)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(parameters)
    ax.set_xlabel('Impacto en LCOE (%)')
    ax.set_title('Análisis de sensibilidad LCOE - Todas las localidades')
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels, loc='upper right', bbox_to_anchor=(1.3, 1))
    ax.grid(True, axis='x')
    plt.tight_layout()
    plt.savefig(RESULTADOS_PV_DIR / 'tornado_analysis_combined.png', bbox_inches='tight', dpi=300)
    plt.close()

# =============================
# 🚀 Ejecución principal
# =============================
def main():
    """
    Función principal: ejecuta la simulación, análisis de sensibilidad y guarda resultados y gráficos.
    """
    sensibilidad_lcoe = []
    sensibilidad_npv = []
    system_capacity_kw = 50000  # 50 MW
    locations = [
        {"name": "Calama", "solar_resource": "/home/nicole/UA/prueba2/datos_limpios/calama_TMY_final.csv"},
        {"name": "Salvador", "solar_resource": "/home/nicole/UA/prueba2/datos_limpios/salvador_TMY_final.csv"},
        {"name": "Vallenar", "solar_resource": "/home/nicole/UA/prueba2/datos_limpios/vallenar_TMY_final.csv"}
    ]
    results = []
    all_results = {}
    hourly_results = []
    for loc in locations:
        print(f"\nProcesando {loc['name']}...")
        annual_energy, lcoe, incident_energy, hourly_power = simulate_pv_plant(
            loc["solar_resource"],
            system_capacity_kw,
            loc["name"]
        )
        df_tmy = pd.read_csv(loc["solar_resource"], skiprows=2)
        datetimes = pd.to_datetime(df_tmy[['Year', 'Month', 'Day', 'Hour', 'Minute']])
        df_hourly = pd.DataFrame({
            "datetime": datetimes,
            "Location": loc["name"],
            "AC Power (kW)": hourly_power
        })
        hourly_results.append(df_hourly)
        npv = calculate_npv(
            annual_energy, 
            system_capacity_kw, 
            spot_price=0.12,  # Cambia este valor si tienes un precio de venta diferente
            fixed_charge_rate=0.08,
            project_lifetime=25,
            capex_pv=800,
            fixed_om_cost=50,
            variable_om_cost=0.01,
            inverter_lifetime=10,
            system_losses=14.0
        )
        print(f"VAN para {loc['name']}: ${npv:,.2f}")
        parameters_npv, variations_npv, impacts_npv = run_npv_sensitivity_analysis(
            annual_energy, system_capacity_kw, npv, loc["name"], spot_price=0.06
        )
        plot_npv_tornado(parameters_npv, impacts_npv, loc["name"])
        for param, (impact_low, impact_high) in zip(parameters_npv, impacts_npv):
            sensibilidad_npv.append({
                "Location": loc["name"],
                "Parameter": param,
                "Impact Low (%)": impact_low,
                "Impact High (%)": impact_high
        })
        parameters, variations, impacts = run_sensitivity_analysis(
            annual_energy, system_capacity_kw, lcoe, loc["name"]
        )
        for param, (impact_low, impact_high) in zip(parameters, impacts):
            sensibilidad_lcoe.append({
                "Location": loc["name"],
                "Parameter": param,
                "Impact Low (%)": impact_low,
                "Impact High (%)": impact_high
        })
        all_results[loc["name"]] = {
            "annual_energy": annual_energy,
            "lcoe": lcoe,
            "impacts": impacts
        }
        results.append({
            "Location": loc["name"],
            "Annual Energy (GWh)": annual_energy / 1e6,
            "LCOE ($/kWh)": lcoe,
            "Incident Energy (kWh/m²)": incident_energy,
            "NPV (VAN)": npv
        })
    df_hourly_all = pd.concat(hourly_results, ignore_index=True)
    df_hourly_all.to_csv(RESULTADOS_PV_DIR / "pv_simulation_results_hourly.csv", index=False)
    print("Resultados horarios guardados en pv_simulation_results_hourly.csv")
    df_results = pd.DataFrame(results)
    print("\nResultados de la simulación:")
    print(df_results.to_string(index=False))
    df_results.to_csv(RESULTADOS_PV_DIR / "pv_simulation_results.csv", index=False)
    print("\nResultados guardados en pv_simulation_results.csv")
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 5))
    ax1.bar(df_results["Location"], df_results["Annual Energy (GWh)"])
    ax1.set_xlabel("Localidad")
    ax1.set_ylabel("Energía anual (GWh)")
    ax1.set_title("Producción anual de energía por localidad")
    ax1.grid(True)
    ax2.bar(df_results["Location"], df_results["LCOE ($/kWh)"])
    ax2.set_xlabel("Localidad")
    ax2.set_ylabel("LCOE ($/kWh)")
    ax2.set_title("Costo nivelado de energía (20 años, 8% FCR)")
    ax2.grid(True)
    ax3.bar(df_results["Location"], df_results["Incident Energy (kWh/m²)"])
    ax3.set_xlabel("Localidad")
    ax3.set_ylabel("Energía incidente (kWh/m²)")
    ax3.set_title("Energía incidente anual por localidad")
    ax3.grid(True)
    plt.tight_layout()
    plt.savefig(RESULTADOS_PV_DIR / "pv_simulation_results.png")
    plt.close()
    plot_combined_tornado(all_results)
    pd.DataFrame(sensibilidad_lcoe).to_csv(RESULTADOS_PV_DIR / "sensibilidad_lcoe.csv", index=False)
    pd.DataFrame(sensibilidad_npv).to_csv(RESULTADOS_PV_DIR / "sensibilidad_npv.csv", index=False)
    print("\nGráfico tornado combinado guardado como 'tornado_analysis_combined.png'")

if __name__ == "__main__":
    main()


Procesando Calama...

Análisis de sensibilidad para Calama:
Producción anual base: 79.28 GWh
LCOE base: 0.0556 $/kWh

FCR:
  Variación baja: 0.06 -> LCOE: 0.0506 $/kWh (Impacto: -9.0%)
  Variación alta: 0.1 -> LCOE: 0.0427 $/kWh (Impacto: -23.3%)

CapEx PV:
  Variación baja: 800 -> LCOE: 0.0461 $/kWh (Impacto: -17.1%)
  Variación alta: 1200 -> LCOE: 0.0578 $/kWh (Impacto: 4.0%)

Spot Price:
  Variación baja: 0.005 -> LCOE: 0.0440 $/kWh (Impacto: -21.0%)
  Variación alta: 0.015 -> LCOE: 0.0482 $/kWh (Impacto: -13.3%)

Inverter Lifetime:
  Variación baja: 8 -> LCOE: 0.0483 $/kWh (Impacto: -13.2%)
  Variación alta: 12 -> LCOE: 0.0457 $/kWh (Impacto: -17.8%)

System Losses:
  Variación baja: 12.0 -> LCOE: 0.0452 $/kWh (Impacto: -18.8%)
  Variación alta: 16.0 -> LCOE: 0.0471 $/kWh (Impacto: -15.3%)
VAN para Calama: $6,595,534.24

Análisis de sensibilidad VAN para Calama:
VAN base: $6,595,534.24

FCR:
  Variación baja: 0.06 -> VAN: $-45,157,200.75 (Impacto: -784.7%)
  Variación alta: 0.1 ->

La simulación fue ejecutada exitosamente para cada ubicación. Se utilizaron los datos meteorológicos limpios y se obtuvieron resultados horarios consistentes. La energía anual obtenida para cada localidad se utilizará en las siguientes secciones para el cálculo de KPIs económicos como LCOE y VAN.
Cabe destacar que Vallenar entregó un VAN muy negativo.

## 3. Dashboard Interactivo

In [ ]:
from dash import Dash, dcc, html, dash_table, Input, Output
import plotly.express as px
import pandas as pd
from pathlib import Path
import plotly.graph_objects as go

# Carpetas
DATOS_LIMPIOS_DIR = Path("datos_limpios")
RESULTADOS_PV_DIR = Path("resultados_pv")

# Carga datos horarios y generales
df_hourly = pd.read_csv(RESULTADOS_PV_DIR / "pv_simulation_results_hourly.csv", parse_dates=["datetime"])
df_kpi = pd.read_csv(RESULTADOS_PV_DIR / "pv_simulation_results.csv")
df_sens_lcoe = pd.read_csv(RESULTADOS_PV_DIR / "sensibilidad_lcoe.csv")
df_sens_npv = pd.read_csv(RESULTADOS_PV_DIR / "sensibilidad_npv.csv")

# Si no tienes columna Year, extráela del datetime
if "Year" not in df_hourly.columns:
    df_hourly["Year"] = df_hourly["datetime"].dt.year

localidades = df_hourly["Location"].unique()
anios = df_hourly["Year"].unique()

app = Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard Solar Integrado"),
    html.Div([
        html.Label("Selecciona país/localidad:"),
        dcc.Dropdown(
            id="dropdown-localidad",
            options=[{"label": loc, "value": loc} for loc in localidades],
            value=localidades[0]
        ),
        html.Label("Selecciona año:"),
        dcc.Dropdown(
            id="dropdown-anio",
            options=[{"label": str(a), "value": a} for a in anios],
            value=anios[0]
        ),
    ], style={"display": "flex", "gap": "2em"}),
    html.H2("Curva horaria de potencia (AC Power)"),
    dcc.Graph(id="grafico-potencia"),
    html.H2("KPIs diarios"),
    html.Div(id="kpi-diarios", style={"display": "flex", "gap": "3em"}),
    html.H2("Datos TMY limpios (primeras filas)"),
    dash_table.DataTable(id="tabla-tmy", page_size=10, style_table={'overflowX': 'auto'}),
    html.H2("Gráficos de radiación horaria"),
    dcc.Graph(id="grafico-tmy"),
    html.H2("Resultados de Simulación PV"),
    dash_table.DataTable(id="tabla-resultados", page_size=5, style_table={'overflowX': 'auto'}),
    html.H2("Análisis de Sensibilidad LCOE"),
    dcc.Graph(id="grafico-tornado-lcoe"),
    dash_table.DataTable(id="tabla-sens-lcoe", page_size=10, style_table={'overflowX': 'auto'}),
    html.H2("Análisis de Sensibilidad NPV"),
    dcc.Graph(id="grafico-tornado-npv"),
    dash_table.DataTable(id="tabla-sens-npv", page_size=10, style_table={'overflowX': 'auto'}),
])
def crear_tornado(df, parametro_impacto_bajo, parametro_impacto_alto, titulo):
    # Ordenar por el mayor impacto absoluto
    df = df.copy()
    df['max_impact'] = df[[parametro_impacto_bajo, parametro_impacto_alto]].abs().max(axis=1)
    df = df.sort_values('max_impact', ascending=True)
    fig = go.Figure()
    fig.add_trace(go.Bar(
        y=df['Parameter'],
        x=df[parametro_impacto_bajo],
        orientation='h',
        name='Impacto bajo',
        marker_color='steelblue'
    ))
    fig.add_trace(go.Bar(
        y=df['Parameter'],
        x=df[parametro_impacto_alto],
        orientation='h',
        name='Impacto alto',
        marker_color='indianred'
    ))
    fig.update_layout(
        barmode='overlay',
        title=titulo,
        xaxis_title='Impacto (%)',
        yaxis_title='Parámetro',
        template='plotly_white',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    return fig
@app.callback(
    Output("grafico-potencia", "figure"),
    Output("kpi-diarios", "children"),
    Output("tabla-tmy", "data"),
    Output("tabla-tmy", "columns"),
    Output("grafico-tmy", "figure"),
    Output("tabla-resultados", "data"),
    Output("tabla-resultados", "columns"),
    Output("grafico-tornado-lcoe", "figure"),
    Output("tabla-sens-lcoe", "data"),
    Output("tabla-sens-lcoe", "columns"),
    Output("grafico-tornado-npv", "figure"),
    Output("tabla-sens-npv", "data"),
    Output("tabla-sens-npv", "columns"),
    Input("dropdown-localidad", "value"),
    Input("dropdown-anio", "value")
)
def actualizar_dashboard(localidad, anio):
    # --- Potencia horaria y KPIs ---
    df_sel = df_hourly[(df_hourly["Location"] == localidad) & (df_hourly["Year"] == anio)].copy()
    fig_pot = px.line(df_sel, x="datetime", y="AC Power (kW)", title=f"Potencia horaria - {localidad} {anio}")
    df_sel["date"] = df_sel["datetime"].dt.date
    energia_diaria = df_sel.groupby("date")["AC Power (kW)"].sum()
    energia_prom = energia_diaria.mean()
    energia_total = energia_diaria.sum()
    lcoe = df_kpi[df_kpi["Location"] == localidad]["LCOE ($/kWh)"].values[0]
    potencia_nominal = df_sel["AC Power (kW)"].max()
    cf_diario = energia_diaria / (potencia_nominal * 24)
    cf_prom = cf_diario.mean()
    kpis = [
        html.Div([
            html.H3("Energía diaria promedio"),
            html.P(f"{energia_prom:.2f} kWh/día")
        ]),
        html.Div([
            html.H3("LCOE"),
            html.P(f"{lcoe:.4f} $/kWh")
        ]),
        html.Div([
            html.H3("Factor de Capacidad promedio"),
            html.P(f"{cf_prom*100:.2f} %")
        ]),
    ]
    # --- TMY limpio y radiación ---
    tmy_file = DATOS_LIMPIOS_DIR / f"{localidad.lower()}_TMY_final.csv"
    df_tmy = pd.read_csv(tmy_file, skiprows=2)
    tmy_data = df_tmy.head(20).to_dict("records")
    tmy_columns = [{"name": i, "id": i} for i in df_tmy.columns]
    fig_tmy = px.line(df_tmy, x=range(len(df_tmy)), y=["GHI", "DNI", "DHI"], labels={"value": "W/m²", "variable": "Componente"}, title=f"Radiación horaria - {localidad}")
    # --- Resultados PV ---
    df_res = df_kpi[df_kpi["Location"] == localidad]
    res_data = df_res.to_dict("records")
    res_columns = [{"name": i, "id": i} for i in df_res.columns]
    # --- Sensibilidad LCOE ---
    df_lcoe = df_sens_lcoe[df_sens_lcoe["Location"] == localidad]
    fig_lcoe = crear_tornado(df_lcoe, "Impact Low (%)", "Impact High (%)", f"Tornado LCOE - {localidad}")
    lcoe_data = df_lcoe.to_dict("records")
    lcoe_columns = [{"name": i, "id": i} for i in df_lcoe.columns]
    # --- Sensibilidad NPV ---
    df_npv = df_sens_npv[df_sens_npv["Location"] == localidad]
    fig_npv = crear_tornado(df_npv, "Impact Low (%)", "Impact High (%)", f"Tornado NPV - {localidad}")    
    npv_data = df_npv.to_dict("records")
    npv_columns = [{"name": i, "id": i} for i in df_npv.columns]
    return (fig_pot, kpis, tmy_data, tmy_columns, fig_tmy, res_data, res_columns, fig_lcoe, lcoe_data, lcoe_columns, fig_npv, npv_data, npv_columns)

app.run(mode='inline')

## 8. Conclusiones
- Salvador tiene el mejor desempeño económico (mayor VAN, bajo LCOE).
- Calama también es competitivo, con potencial de mejora mediante reducción de CapEx o pérdidas.
- Vallenar no es viable bajo las condiciones actuales, con VAN negativo y bajo rendimiento.
- El análisis de sensibilidad muestra fuerte dependencia del modelo económico al precio spot y la inversión.
- El dashboard permite explorar dinámicamente los resultados por país y año.